# TensorFlow Debugger (tfdbg) Command-Line-Interface Tutorial: Tutorial 

**(Experimental)**

这一节学习使用tfdbg，在TensorFlow的使用过程中很常见的错误是bad numerical values(nan和inf)，从而导致训练失败。我们看一下碰到这种错误，如何用tfdbg一步一步找到问题并解决。

 先执行一个示例程序：

In [ ]:
python debug_mnist.py

Extracting MNIST_data/train-images-idx3-ubyte.gz

Extracting MNIST_data/train-labels-idx1-ubyte.gz

Extracting MNIST_data/t10k-images-idx3-ubyte.gz

Extracting MNIST_data/t10k-labels-idx1-ubyte.gz

Accuracy at step 0: 0.1113

Accuracy at step 1: 0.3183

Accuracy at step 2: 0.098

Accuracy at step 3: 0.098

Accuracy at step 4: 0.098

Accuracy at step 5: 0.098

Accuracy at step 6: 0.098

Accuracy at step 7: 0.098

Accuracy at step 8: 0.098

Accuracy at step 9: 0.098

可以看到准确率先升然后迅速下降。通常问题是因为某个节点产生了inf或nan。

## Wrapping TensorFlow Sessions with tfdbg

只需要修改少量的代码，就能为Session添加一层调试器，同时，在terminal运行代码时需要加上参数 `--debug`:

In [ ]:
from tensorflow.python import debug as tf_debug

sess = tf_debug.LocalCLIDebugWrapperSession(sess)
sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)

wrapper的接口和Session一样，wrapper额外的优点有：
* 在每次调用run()开始前和结束后，都会绑定一个基于terminal 的UI界面，来让你控制计算图内部的执行，检测内部节点的状态
* 允许你注册特殊的过滤器来过滤tensor值，方便诊断问题所在

上面的代码我们注册了一个tensor过滤器 `tfdbg.has_inf_or_nan`，它会查找计算图的内部tensor是否含有nan或inf

In [3]:
def has_inf_or_nan(datum, tensor):
    return np.any(np.isnan(tensor)) or np.any(np.isinf(tensor))

**TIPS** 你也可以实现自己的过滤器，对这部分感兴趣的童鞋去查看`DebugDumpDir.find()`的API吧。

## Debugging Model Training with tfdbg

我们重新执行最开始的训练代码并且开启调试。

In [ ]:
python debug_mnist.py --debug

第一次调用`run()`之前，会进入下面的UI界面：

![](http://odw1x7kgr.bkt.clouddn.com/WX20170307-105846@2x.png)

我们把这个界面称为 run-start UI。

如果你嫌这个界面太小，可以用**PageUp/PageDown/Home/End**键来调整界面的大小。

常用的命令：

pt-------print tensor

ni-------node information

li-------list input

lo-------list output

r--------run


![](http://odw1x7kgr.bkt.clouddn.com/WX20170307-111145.png)